<a href="https://colab.research.google.com/github/revanthbethala/shoreline_extraction/blob/main/gee_satellite_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import os

# ============================================
# Initialize Earth Engine
# ============================================
try:
    ee.Authenticate()
    ee.Initialize(project="ee-revanthbethala")
    print("✅ Earth Engine initialized successfully!")
except Exception as e:
    print(f"❌ Earth Engine initialization failed: {e}")

# ============================================
# Shoreline Image Exporter (All Images)
# ============================================
class ShorelineDLExporter:
    def __init__(self, study_area_geometry, output_dir="shoreline_dl_data"):
        self.study_area = study_area_geometry
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)

        # Satellite collections
        self.collections = {
            "landsat5": {
                "collection": "LANDSAT/LT05/C02/T1_L2",
                "start_year": 1984,
                "end_year": 2013,
                "bands": {"blue":"SR_B1","green":"SR_B2","red":"SR_B3","nir":"SR_B4","swir1":"SR_B5","swir2":"SR_B7"},
                "qa_band":"QA_PIXEL"
            },
            "landsat7": {
                "collection": "LANDSAT/LE07/C02/T1_L2",
                "start_year": 1999,
                "end_year": 2024,
                "bands": {"blue":"SR_B1","green":"SR_B2","red":"SR_B3","nir":"SR_B4","swir1":"SR_B5","swir2":"SR_B7"},
                "qa_band":"QA_PIXEL"
            },
            "landsat8": {
                "collection": "LANDSAT/LC08/C02/T1_L2",
                "start_year": 2013,
                "end_year": 2024,
                "bands": {"blue":"SR_B2","green":"SR_B3","red":"SR_B4","nir":"SR_B5","swir1":"SR_B6","swir2":"SR_B7"},
                "qa_band":"QA_PIXEL"
            },
            "landsat9": {
                "collection": "LANDSAT/LC09/C02/T1_L2",
                "start_year": 2021,
                "end_year": 2024,
                "bands": {"blue":"SR_B2","green":"SR_B3","red":"SR_B4","nir":"SR_B5","swir1":"SR_B6","swir2":"SR_B7"},
                "qa_band":"QA_PIXEL"
            },
            "sentinel2": {
                "collection": "COPERNICUS/S2_SR_HARMONIZED",
                "start_year": 2015,
                "end_year": 2024,
                "bands": {"blue":"B2","green":"B3","red":"B4","nir":"B8","swir1":"B11","swir2":"B12"},
                "qa_band":"QA60"
            }
        }

        self.feature_bands = ["blue","green","red","nir","swir1","swir2"]

    # ---------------------------
    # Harmonization and scaling
    # ---------------------------
    def _harmonize_and_scale(self, image, sat_name):
        bands = self.collections[sat_name]["bands"]
        common_names = list(self.feature_bands)

        if "sentinel" in sat_name:
            scale_factor = 0.0001
            offset = 0
        else:
            scale_factor = 0.0000275
            offset = -0.2

        return image.select(list(bands.values()), common_names).multiply(scale_factor).add(offset).toFloat()

    # ---------------------------
    # Cloud masking
    # ---------------------------
    def _mask_clouds(self, image, sat_name):
        qa_band = self.collections[sat_name]["qa_band"]
        qa = image.select(qa_band)

        if "sentinel" in sat_name:
            cloud_bit_mask = 1 << 10
            cirrus_bit_mask = 1 << 11
            mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        else:
            cloud_shadow_bit = 1 << 3
            cloud_bit = 1 << 5
            mask = qa.bitwiseAnd(cloud_shadow_bit).eq(0).And(qa.bitwiseAnd(cloud_bit).eq(0))

        return image.updateMask(mask)

    # ---------------------------
    # Export all images individually
    # ---------------------------
    def export_all_images(self, start_year, end_year, max_cloud=20):
        print("🚀 Exporting all individual images...")

        for year in range(start_year, end_year + 1):
            year_start = ee.Date.fromYMD(year, 1, 1)
            year_end = year_start.advance(1, "year")
            merged_collection = ee.ImageCollection([])

            # Merge all available satellites for the year
            for sat_name, sat_info in self.collections.items():
                if sat_info["start_year"] <= year <= sat_info["end_year"]:
                    collection = (
                        ee.ImageCollection(sat_info["collection"])
                        .filterBounds(self.study_area)
                        .filterDate(year_start, year_end)
                    )
                    cloud_property = "CLOUDY_PIXEL_PERCENTAGE" if "sentinel" in sat_name else "CLOUD_COVER"
                    collection = collection.filter(ee.Filter.lt(cloud_property, max_cloud))

                    def process_image(img):
                        cloud_masked = self._mask_clouds(img, sat_name)
                        harmonized = self._harmonize_and_scale(cloud_masked, sat_name)
                        return harmonized.set("system:time_start", img.get("system:time_start")).set("sensor", sat_name)

                    merged_collection = merged_collection.merge(collection.map(process_image))

            count = merged_collection.size().getInfo()
            if count == 0:
                print(f"⚠️ {year}: No images found.")
                continue

            print(f"📅 {year}: {count} images found. Starting export...")

            # Convert to list for exporting
            image_list = merged_collection.toList(count)
            for i in range(count):
                img = ee.Image(image_list.get(i))
                sensor = img.get("sensor").getInfo()
                timestamp = img.get("system:time_start").getInfo()
                file_name = f"{year}_{sensor}_{timestamp}"

                task = ee.batch.Export.image.toDrive(
                    image=img,
                    description=f"Export_{file_name}",
                    folder=f"{year}",
                    fileNamePrefix=file_name,
                    region=self.study_area.bounds(),
                    scale=10,
                    crs="EPSG:32644",
                    fileFormat="GeoTIFF",
                    maxPixels=1e13,
                )
                task.start()

        print("\n✅ All images export tasks started successfully!")

# ============================================
# Main Execution
# ============================================
def main():
    baseline = geemap.geojson_to_ee("/content/test.geojson")
    study_area = baseline.geometry().buffer(1000)

    exporter = ShorelineDLExporter(study_area)
    exporter.export_all_images(
        start_year=1990,
        end_year=2025,
        max_cloud=20
    )

if __name__ == "__main__":
    main()